In [ ]:
%matplotlib inline
from __future__ import print_function
import errno
import datetime
import gc
import gzip
import math
import os
import random
import sys
import traceback
import sklearn.metrics
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import ipywidgets
from IPython.display import Image
from IPython.display import display
from six.moves import cPickle as pickle

import improc
import convnet
import mutate
import convevo
import darwin

In [ ]:
reload (improc)
reload (convnet)
reload (mutate)
reload (convevo)
reload (darwin)

In [ ]:
# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

# Enumerate Images
Image names are sequential, so add every tenth image to the validation set based on filename.

In [ ]:
training = []
test = []

for root, dirs, files in os.walk('captures'):
    for name in files:
        path = os.path.join(root, name)
        low_name = name.lower()
        # Find all the image files, split into test and training.
        if low_name.endswith(".png"):
            if low_name.endswith("0.png"):
                test.append(path)
            else:
                training.append(path)

print("Training:", len(training), "Test:", len(test))
print(training[:2])
print(test[:2])

# Image Processing
Each image file contains a color image (top half), and an encoded depth image (bottom half)
<img src="testing/IMG_2114.PNG">
* Note: The image may also contain the orientation data. If so it is encoded in the first two pixels of the depth image. If the first pixel of the depth image is red, the second has the x, y, z, w quaternion components encoded in the r,g,b,a values.

The improc module contains functions for splitting the image, decoding the depth back into floating point millimeters, and for filling in gaps.

## Image processing examples:

In [ ]:
example_image, example_depth, example_attitude = improc.load_image("testing/IMG_2114.PNG")
plt.imshow(example_image)
print(example_image.shape, example_image.dtype)

In [ ]:
plt.imshow(example_depth)
print(example_depth.shape, example_depth.dtype)
print(example_attitude)

In [ ]:
example_lab = improc.rgb2lab_normalized(example_image)
plt.imshow(example_lab[:,:,0], cmap='Greys_r')

In [ ]:
plt.imshow(example_lab[:,:,1], cmap='Greys_r')

# Depth Labels and Batching

In [ ]:
def compute_average_depth():
    depth_averages = []

    for path in training:
        _, depth, _ = improc.load_image(path)
        depth_averages.append(np.nanmean(depth))
        if len(depth_averages) % 1000 == 0:
            print("Image", len(depth_averages))
    return np.nanmean(depth_averages)

# Precomputed via compute_average_depth()
MEAN_DEPTH = np.float32(1688.97)

print(MEAN_DEPTH)

## Covert depth to classification labels.
Want more precision for nearby things, so use progressively expanding buckets for labels, so if smallest bucket has size s and each succesive bucket is larger by a factor F then:

improc.MAX_DEPTH == sF<sup>0</sup> + sF<sup>1</sup> + sF<sup>2</sup> + ... + sF<sup>label count - 1</sup>

So, plug into sum of geometric series formula:

improc.MAX_DEPTH == s * (1 - F<sup>label count</sup>) / (1 - F)

Since there are two unknowns we can choose either the factor or the bucket size. A factor of 1.3 resulted in buckets that seemed about right.

In [ ]:
def size_for_factor(factor, buckets):
    return improc.MAX_DEPTH * (1 - factor) / (1 - factor ** buckets)

def depth_label_boundaries(factor, buckets):
    boundaries = []
    size_sum = 0
    bucket_size = size_for_factor(factor, buckets)
    for i in range(buckets):
        size_sum += bucket_size
        boundaries.append(size_sum)
        bucket_size *= factor
    return boundaries

DEPTH_LABEL_COUNT = 20
DEPTH_BUCKET_SCALE_FACTOR = 1.3
DEPTH_BOUNDARIES = [(i + 1) * improc.MAX_DEPTH / DEPTH_LABEL_COUNT for i in xrange(DEPTH_LABEL_COUNT)]
print(DEPTH_BOUNDARIES[:5])

def depth_label_index(depth):
    for i, boundary in enumerate(DEPTH_BOUNDARIES):
        if depth < boundary:
            return i
    return DEPTH_LABEL_COUNT - 1

def depth_label(depth, labels=None):
    if labels is None:
        labels = np.zeros(shape=(DEPTH_LABEL_COUNT + 1), dtype=np.float32)
    labels[depth_label_index(depth)] = 1
    labels[DEPTH_LABEL_COUNT] = depth / improc.MAX_DEPTH
    return labels

def depth_label_image(depths):
    labeled = depths.copy()
    for y in xrange(depths.shape[0]):
        for x in xrange(depths.shape[1]):
            labeled[y,x] = depth_label_index(depths[y,x])
    return labeled

print("Mean depth label:", depth_label(MEAN_DEPTH))
print("Zero depth label:", depth_label(0)[0], depth_label(0)[-1])
print("Max depth label:", depth_label(improc.MAX_DEPTH)[-2:])

In [ ]:
def fisher_yates_shuffle(items, entropy):
    """Based on https://en.wikipedia.org/wiki/Fisher%E2%80%93Yates_shuffle"""
    i = len(items)
    while i > 1:
        i = i - 1
        j = entropy.randint(0, i)
        items[j], items[i] = items[i], items[j]

In [ ]:
# Set up cache directory.
depth_image_cache_path = os.path.join("temp", "cache")
try:
    os.makedirs(depth_image_cache_path)
except OSError as e:
    pass

class ImageSampler(object):
    """Wrap an image for sampling."""
    def __init__(self, image_file, sample_height, sample_width, half_valid_check=2, tolerance=0):
        # See if the image has already been processed and cached
        cached = None
        cache_path = os.path.join(depth_image_cache_path, os.path.split(image_file)[1]) + ".pickle"

        try:
            with gzip.open(cache_path, 'rb') as f:
                cached = pickle.load(f)
        except KeyboardInterrupt:
            raise
        except OSError as e:
            if e.errno != errno.ENOENT:
                print("OSError opening cached image:", e.errno, e) 
        except IOError as e:
            if e.errno != errno.ENOENT:
                print("IOError opening cached image:", e.errno, e) 
        except Exception as e:
            print("Error opening cached image:", e)
            
        if cached:
            self.image = cached["image"]
            self.depth = cached["depth"]
        else:
            # Load and process the image.
            self.image, self.depth, _ = improc.load_image(image_file)
            # Convert from rgb to CIE LAB format.
            self.image = improc.rgb2lab_normalized(self.image)
        
        if not cached:
            try:
                with gzip.open(cache_path, 'wb') as f:
                    cache_data = { "image": self.image, "depth": self.depth}
                    pickle.dump(cache_data, f, pickle.HIGHEST_PROTOCOL)
            except KeyboardInterrupt:
                raise
            except Exception as e:
                print("Error caching image:", image_file, "-", e)
        self.index = 0
        self.pixel_index = (0, 0)
        self.sample_height = sample_height
        self.sample_width = sample_width
        self.depth_offset_y = (sample_height + 1) / 2
        self.depth_offset_x = (sample_width + 1) / 2
        self.height = self.image.shape[0]        
        self.width = self.image.shape[1]
        self.half_valid_check = half_valid_check
        self.tolerance = tolerance
        
    def depth_value(self, y, x):
        return self.depth[y + self.depth_offset_y, x + self.depth_offset_x]
        
    def sample(self, inputs, labels, index):
        y, x = self.pixel_index
        patch = self.image[y : y + self.sample_height, x : x + self.sample_width]
        inputs[index] = patch
        depth_label(self.depth_value(y, x), labels[index])
        self.advance()

    def setup_sample_order(self, sample_orders, entropy):
        height_span = self.height - self.sample_height
        width_span = self.width - self.sample_width
        cached = sample_orders.get((height_span, width_span))
        if cached:
            return cached

        pixel_indices = []
        for y in range(height_span):
            for x in range(width_span):
                pixel_indices.append((y, x))
                
        fisher_yates_shuffle(pixel_indices, entropy)
        sample_orders[(height_span, width_span)] = pixel_indices
        return pixel_indices
        
    def advance(self):
        self.index += 1
    
    def next_sample(self, sample_orders, entropy):
        c = self.half_valid_check
        order = self.setup_sample_order(sample_orders, entropy)
        while self.index < len(order):
            self.pixel_index = order[self.index]
            depth_y = self.pixel_index[0] + self.depth_offset_y
            depth_x = self.pixel_index[1] + self.depth_offset_x
            # Check that the sample is from a clean part of the image.
            sum = np.sum(np.isnan(self.depth[depth_y - c : depth_y + c, depth_x - c: depth_x + c]))
            if sum <= self.tolerance:
                return True
            self.advance()
        return False

In [ ]:
class BatchSampler(object):
    """Created sample batches for a set of image files"""
    def __init__(self, image_files, sample_height, sample_width, samplers_count=100):
        self.files = image_files
        self.samplers_count = samplers_count
        self.sample_height = sample_height
        self.sample_width = sample_width
        self.sample_orders = {}
        self.reset()
    
    # Access or initialize the specified sampler.
    def sampler(self, index, entropy):
        sampler = self.samplers[index]
        if sampler and not sampler.next_sample(self.sample_orders, entropy):
            sampler = None

        while sampler is None:
            path = self.files[self.file_index]
            sampler = ImageSampler(path, self.sample_height, self.sample_width)
            self.file_index = (self.file_index + 1) % len(self.files)
            if not sampler.next_sample(self.sample_orders, entropy):
                sampler = None
                print ("No samples in", path)
            else:
                self.samplers[index] = sampler
        return sampler
    
    # Get the next single sample.
    def sample(self, inputs, labels, index, entropy):
        sampler = self.sampler(self.sample_index, entropy)

        self.sample_index = (self.sample_index + 1) % len(self.samplers)
        sampler.sample(inputs, labels, index)
    
    # Get the next batch of samples.
    def sample_batch(self, inputs, labels, batch_size, entropy):
        labels.fill(0)
        for b in xrange(batch_size):
            self.sample(inputs, labels, b, entropy)
            
    def reset(self):
        self.sample_index = 0
        self.file_index = 0
        self.samplers = [None] * self.samplers_count
    
    # Force load all the samplers.
    def fill_and_pickle(self, path, entropy):
        for i in range(self.samplers_count):
            sampler = self.sampler(i, entropy)

        try:
            with open(path, 'wb') as f:
                pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', path, ':', e)
            raise

## Depth label and batching examples

In [ ]:
plt.imshow(depth_label_image(example_depth))

In [ ]:
del example_image
del example_depth
del example_lab
gc.collect()

In [ ]:
SAMPLE_SIZE = 101
batcher = BatchSampler(["testing/IMG_2114.PNG", "testing/IMG_3410.PNG"], SAMPLE_SIZE, SAMPLE_SIZE, 2)

In [ ]:
BATCH_SIZE = 100

inputs = np.ones(shape=(BATCH_SIZE, SAMPLE_SIZE, SAMPLE_SIZE, improc.COLOR_CHANNELS), dtype=np.float32)
labels = np.zeros(shape=(BATCH_SIZE, DEPTH_LABEL_COUNT + 1), dtype=np.float32)

for _ in xrange(100):
    batcher.sample_batch(inputs, labels, BATCH_SIZE, random.Random(42))

In [ ]:
plt.imshow(inputs[1,:,:,0], cmap='Greys_r')
print(inputs[1].shape)
print(labels[1])

# Data Management

In [ ]:
data_files = {
    "image_size": (101, 101, improc.COLOR_CHANNELS),
    "depth_labels": DEPTH_LABEL_COUNT,
    "train_files": np.array(training),
    "test_files": np.array(test)
}

del training
del test

In [ ]:
def setup_cross_validation(data, train_count, valid_count, test_count=None, label_count=None, entropy=random):
    """Shuffle the data and split off training, validation and test sets."""
    cross_data = data.copy()
    
    if label_count:
        cross_data["depth_labels"] = label_count

    paths = cross_data["train_files"][:]
    fisher_yates_shuffle(paths, entropy)

    cross_data["train_files"] = paths[:train_count]
    cross_data["valid_files"] = paths[train_count:train_count + valid_count]

    if test_count is not None:
        cross_data["test_files"] = data["test_files"][:test_count]

    return cross_data

## Batcher Caching
The evolutionary process will involve running many graphs with the same data. To make this as efficent as possible, these are used cache and restore the processed batch data.

In [ ]:
def pickle_batch(data, set_name, samplers, entropy):
    path = os.path.join("temp", set_name + ".pickle")
    files = data[set_name + "_files"]
    image_size = data["image_size"]
    batcher = BatchSampler(files, image_size[0], image_size[1], samplers)
    batcher.fill_and_pickle(path, entropy)
    del batcher
    gc.collect()
    return path

def load_batcher(pickle_batches, set_name):
    if pickle_batches:
        path = pickle_batches.get(set_name)
        if path:
            with open(path, 'rb') as f:
                return pickle.load(f)
    return None

## Data Management examples

In [ ]:
pickle_data = setup_cross_validation(
    data_files, 0, 100, None,
    label_count=DEPTH_LABEL_COUNT, entropy=random.Random(24601)
)
pickle_size = pickle_data["image_size"]
pickle_files = pickle_data["valid_files"]
pickle_sampler = BatchSampler(pickle_files, pickle_size[0], pickle_size[1], len(pickle_files))

In [ ]:
pickle_sampler.fill_and_pickle("temp/depth_valid.pickle", random)

In [ ]:
with open("temp/depth_valid.pickle", 'rb') as f:
    loaded_sampler = pickle.load(f)

In [ ]:
BATCH_SIZE = 100

inputs = np.ones(shape=(BATCH_SIZE, pickle_size[0], pickle_size[1], improc.COLOR_CHANNELS), dtype=np.float32)
labels = np.zeros(shape=(BATCH_SIZE, DEPTH_LABEL_COUNT + 1), dtype=np.float32)

for _ in xrange(500):
    loaded_sampler.sample_batch(inputs, labels, BATCH_SIZE, random.Random(42))

In [ ]:
del pickle_data
del pickle_files
del pickle_sampler
del loaded_sampler
gc.collect()

# Graph Setup

In [ ]:
def batch_input_shape(batch_size, image_shape):
    return (batch_size,) + image_shape

def setup_graph(
    batch_size,
    image_shape,
    label_count,
    regress_factor,
    layer_stack
):
    graph = tf.Graph()
    with graph.as_default():
        input_shape = batch_input_shape(batch_size, image_shape)
        output_shape = (batch_size, label_count + 1)
        train   = tf.placeholder(tf.float32, shape=input_shape)
        targets = tf.placeholder(tf.float32, shape=output_shape)
        verify  = tf.placeholder(tf.float32, shape=input_shape)

        layers = layer_stack.construct(input_shape)
        l2_loss = 0
        
        for layer in layers:
            layer.setup_parameters()
            l2_loss = layer.update_loss(l2_loss)
        
        def model(nodes, train):
            for layer in layers:
                nodes.append(layer.connect(nodes[-1], train))
            return nodes[-1]

        result = model([train], True)
        
        depth_label = tf.slice(targets, [0, label_count], [batch_size, 1])
        depths      = tf.slice(result, [0, label_count], [batch_size, 1])
        labels = tf.slice(targets, [0, 0], [batch_size, label_count])
        logits = tf.slice(result, [0, 0], [batch_size, label_count])

        global_step = tf.Variable(1)
        
        loss = l2_loss
        if regress_factor >= 0:
            loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
        else:
            regress_factor = -regress_factor

        if regress_factor > 0:
            loss += regress_factor * tf.reduce_mean(tf.squared_difference(depths, depth_label))
        
        optimizer = layer_stack.construct_optimizer(global_step)
        
        verify_result = model([verify], False)
        verify_logits = tf.slice(verify_result, [0, 0], [batch_size, label_count])
        verify_depths = tf.slice(verify_result, [0, label_count], [batch_size, 1])

        verify_depths = tf.maximum(verify_depths, -1)
        verify_depths = tf.minimum(verify_depths, 1)
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "targets": targets,
            "depths": depths,
            "loss": loss,
            "optimizer": optimizer.minimize(loss, global_step=global_step),

            # Predictions for training and verification (validation or test)
            "predictions": tf.nn.softmax(logits),
            "verify": verify,
            "verify_predictions": tf.nn.softmax(verify_logits),
            "verify_depths": verify_depths,
            "saver": tf.train.Saver()
        }
    return info

In [ ]:
def setup_save_model(graph_info, path):
    """Set up option to tell run_graph to save the graph after training."""
    graph_info["save_to"] = path
    
def setup_restore_model(graph_info, path):
    """Set up option to tell run_graph to restore graph values."""
    graph_info["restore_from"] = path

# Graph Execution

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [ ]:
def mean_depth_error(depths, labels):
    return np.mean(np.absolute(depths - labels[:,-1]))

In [ ]:
def score_result(loss, predictions, depths, labels):
    return (loss, accuracy(predictions, labels[:,0:-1]), mean_depth_error(depths, labels))

In [ ]:
def print_batch_info(context, score, predictions, depths, labels, verbose=True, print_count=20, depth_print=10):
    print(context, "accuracy: %.1f%%" % score[1])
    if verbose:
        print(np.argmax(predictions[0:print_count],1))
        print(np.argmax(labels[0:print_count,0:-1],1))
        print(context, "average depth error:", score[2])
        print(depths[0:depth_print,0])
        print(labels[0:depth_print,-1])

In [ ]:
def batch_accuracy(context, session, graph_info, batcher, entropy, inputs, labels, batch_size, count, verbose):
    total_accuracy = 0
    total_depth = 0
    for b in xrange(count):
        batcher.sample_batch(inputs, labels, batch_size, entropy)
        targets = [graph_info["verify_predictions"], graph_info["verify_depths"]]
        predictions, depths = session.run(targets, feed_dict={graph_info["verify"] : inputs})
        total_accuracy += accuracy(predictions, labels) / float(count)
        total_depth += mean_depth_error(depths, labels) / float(count)
    
    score = (0, total_accuracy, total_depth)
    print_batch_info(context, score, predictions, depths, labels, verbose)
    return score

In [ ]:
def run_graph(
    graph_info,
    data,
    step_count,
    valid_count,
    test_count=0,
    batch_sampler_count=1000,
    report_every=50,
    verbose=True,
    accuracy_minimum=None, # Minimimum validation percent accuracy for early abort
    pickle_batches=None, # pickle files for training and validation batchers
    progress=None, # A progress bar widget
    tracker=None, # A function for result tracking
    entropy=random
):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()

        # Optionally restore graph parameters from disk.
        restore_from = graph_info.get("restore_from")
        if restore_from:
            graph_info["saver"].restore(session, restore_from)
            print("Restored model from", restore_from)
            
        print("Initialized")

        # Set up space for graph inputs / feed values
        batch_size = graph_info["batch_size"]
        depth_labels = data["depth_labels"]
        height, width, _ = data["image_size"]
        inputs = np.ones(shape=batch_input_shape(batch_size, data["image_size"]), dtype=np.float32)
        labels = np.zeros(shape=(batch_size, depth_labels + 1), dtype=np.float32)

        # Construct or unpickle training batcher.
        train_batcher = load_batcher(pickle_batches, "train")
        if not train_batcher:
            train_batcher = BatchSampler(data["train_files"], height, width, batch_sampler_count)

        score = 0

        try:
            for step in xrange(step_count + 1):
                # Update progress bar, if present
                if progress:
                    progress.value = step

                # Generate a batch and run the graph
                train_batcher.sample_batch(inputs, labels, batch_size, entropy)
                run_targets = [
                    graph_info["optimizer"],
                    graph_info["loss"],
                    graph_info["predictions"],
                    graph_info["depths"]
                ]
                feed_dict = {graph_info["train"] : inputs, graph_info["targets"] : labels}
                _, loss, predictions, depths = session.run(run_targets, feed_dict=feed_dict)
                
                # Keep track of and possibly display score.
                batch_score = score_result(loss, predictions, depths, labels)
                if tracker:
                    tracker(batch_score)

                if np.isnan(loss):
                    print("Error computing loss at step", step)
                    print_batch_info("Minibatch", batch_score, predictions, depths, labels, True)
                    return 0
                if (step % report_every == 0):
                    if verbose:
                        print("Minibatch loss at step", step, ":", loss)
                        print_batch_info("Minibatch", batch_score, predictions, depths, labels, True)

                    # Evaluate the validation data.
                    valid_batcher = load_batcher(pickle_batches, "valid")
                    if not valid_batcher:
                        valid_files = data["valid_files"]
                        valid_batcher = BatchSampler(valid_files, height, width, len(valid_files))
                    valid_score = batch_accuracy(
                        "Validation", session, graph_info, valid_batcher, entropy,
                        inputs, labels, batch_size, valid_count, verbose
                    )
                    del valid_batcher
                    score = valid_score[1]
                    if accuracy_minimum and step > 0 and valid_score[1] < accuracy_minimum:
                        print("Early out.")
                        break

            # Evaluate the test data, if any.
            if test_count > 0:
                test_batcher = BatchSampler(data["test_files"], height, width)
                valid_accuracy = batch_accuracy(
                    "Test", session, graph_info, test_batcher, entropy,
                    inputs, labels, batch_size, test_count, verbose
                )

            return score
        finally:
            # Optionally save out graph parameters to disk.
            save_to = graph_info.get("save_to")
            if save_to:
                graph_info["saver"].save(session, save_to)
                print("Saved model to", save_to)
        

In [ ]:
results_path = os.path.join("temp", "results")
try:
    os.makedirs(results_path)
except OSError as e:
    pass

def save_results(timestamp, results):
    with open(os.path.join(results_path, timestamp + ".csv"), "w") as text_file:
        text_file.write("Loss,Accuracy,Depth Error\n")
        for score in results:
            text_file.write((",".join(str(v) for v in score)) + "\n")
    print("Saved results:", timestamp)

In [ ]:
def make_eval(
    batch_size=20,
    eval_steps=10000,
    valid_steps=500,
    regress_factor=1.0,
    reuse_cross=False,
    metric=None,
    entropy=random
):
    pickle_batches = {}
    train_count = 9700
    valid_count = 400
    batch_sampler_count = min(1000, eval_steps * batch_size)
    test_count = None
    
    #if reusing data, set up training and test data, and pickle batchers for efficiency.
    if reuse_cross:
        redata = setup_cross_validation(
            data_files, train_count, valid_count, test_count,
            label_count=DEPTH_LABEL_COUNT, entropy=entropy
        )
        pickle_batches["valid"] = pickle_batch(redata, "valid", len(redata["valid_files"]), entropy)
        print("Pickled Validation")
        pickle_batches["train"] = pickle_batch(redata, "train", batch_sampler_count, entropy)
        print("Pickled Training")

    # Set up to show a progress bar so you some mesure of time required. Updated in run_graph above.
    progress_bar = ipywidgets.FloatProgress(min=0, max=eval_steps, description="Graph Steps:")
    display(progress_bar)
    
    # Set up to show current training results as well as a running average. updated in record_score below. 
    def setup_label(title):
        return ipywidgets.FloatText(value=0, description=title, disabled=True)
    current_display = [setup_label(title) for title in ["Loss", "Accuracy", "Error"]]
    average_display = [setup_label(" ") for _ in current_display]
    display(ipywidgets.HBox([
        ipywidgets.Box([ipywidgets.HTML("<div style=""margin-left:90px"">Current</div>")] + current_display),
        ipywidgets.Box([ipywidgets.HTML("<div style=""margin-left:90px"">Running Average</div>")] + average_display)
    ]))
  
    def evaluate(stack, entropy):
        # If not reusing data, generate training and validation sets
        if not reuse_cross:
            data = setup_cross_validation(
                data_files, train_count, valid_count, test_count,
                label_count=DEPTH_LABEL_COUNT, entropy=entropy
            )
        else:
            data = redata

        # Set up the Tensorflow graph
        try:
            evo_graph = setup_graph(batch_size, data["image_size"], data["depth_labels"], regress_factor, stack)
        except KeyboardInterrupt:
            raise
        except:
            # Record any errors and the stack that caused them.
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            print(lines[-1])
            convevo.output_error(stack, lines, "temp")
            return -10

        timestamp = datetime.datetime.now().strftime("%Y-%m-%d~%H_%M_%S_%f")
        with open(os.path.join(results_path, timestamp + ".xml"), "w") as text_file:
            text_file.write(convevo.serialize(stack))
        
        setup_save_model(evo_graph, os.path.join(results_path, timestamp + ".ckpt"))
        
        # Record and display the results
        results = []
        def record_score(score):
            results.append(score)
            for display, value in zip(current_display, score):
                display.value = value
            
            resultCount = min(len(results), 100)
            averages = [sum(x)/resultCount for x in zip(*results[-resultCount:])]
            for display, value in zip(average_display, averages):
                display.value = value

        # Run the graph
        try:
            valid_accuracy = run_graph(
                evo_graph,
                data,
                eval_steps,
                valid_count=valid_steps,
                batch_sampler_count=batch_sampler_count,
                report_every=eval_steps/4,
                verbose=True,
                accuracy_minimum=None,
                pickle_batches=pickle_batches,
                progress=progress_bar,
                tracker=record_score,
                entropy=entropy
            )
            if metric:
                return metric(valid_accuracy, results)
            return valid_accuracy
        except KeyboardInterrupt:
            raise
        except:
            # Record any errors and the stack that caused them.
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            print(lines[-1])
            convevo.output_error(stack, lines, "temp")
            return -1
        finally:
            save_results(timestamp, results)
    return evaluate

In [ ]:
def train_accuracy_metric(valid_accuracy, train_results):
    result_count = min(len(train_results), 1000)
    return sum(accuracy for loss, accuracy, depth_error in train_results[-result_count:]) / result_count

In [ ]:
def train_depth_error_metric(valid_accuracy, train_results):
    result_count = min(len(train_results), 1000)
    error = sum(depth_error for loss, accuracy, depth_error in train_results[-result_count:]) / result_count
    return max(0, 1 - error)

In [ ]:
def create_stack(convolutions, flatten, hidden_sizes, output_size, init_mean, init_scale, l2, optimizer=None):
    stack = convevo.LayerStack(flatten=flatten, optimizer=optimizer)
    default_init = lambda: convevo.Initializer("normal", mean=init_mean, scale=init_scale)

    for operation, patch_size, stride, depth, padding, relu in convolutions:
        layer = convevo.ImageLayer(operation, patch_size, stride, depth, "SAME", default_init(), l2_factor=l2)
        stack.add_layer(layer, relu=relu)
    for hidden_size in hidden_sizes:
        layer = convevo.HiddenLayer(hidden_size, bias=True, initializer=default_init(), l2_factor=l2)
        stack.add_layer(layer, relu=True)
    if output_size is not None:
        layer = convevo.HiddenLayer(output_size, bias=True, initializer=default_init(), l2_factor=l2)
        stack.add_layer(layer, relu=False)
    
    return stack

# Test of components in isoloation

In [ ]:
cross_data = setup_cross_validation(data_files, 9700, 400, 1000, label_count=DEPTH_LABEL_COUNT)

In [ ]:
batch_size = 20
conv_layers = [
    ("conv_bias", 20, 2, 10, "SAME", True),
    ("conv_bias", 10, 5, 20, "SAME", True),
    ("conv_bias",  5, 2, 40, "SAME", True)
]
hidden_sizes = [400,100]
optimizer = convevo.Optimizer("GradientDescent", 0.01)
optimizer.default_parameters()
prototype = create_stack(conv_layers, True, hidden_sizes, cross_data["depth_labels"] + 1, 0.0, 0.05, 0.0, optimizer)
prototype.reseed(random.Random(42))

In [ ]:
prototype_graph = setup_graph(batch_size, cross_data["image_size"], cross_data["depth_labels"], 1.0, prototype)

In [ ]:
run_graph(
    prototype_graph, cross_data, 10000,
    valid_count=2000, report_every=1000, verbose=True, entropy=random.Random(42)
)

In [ ]:
print(convevo.serialize(prototype))
prototype_entropy = random.Random(42)
prototype_eval = make_eval(
    batch_size=100, eval_steps=100, valid_steps=20, regress_factor=1.0, reuse_cross=True, entropy=prototype_entropy
)
prototype_eval(prototype, prototype_entropy)

In [ ]:
del cross_data
del conv_layers
del hidden_sizes
del prototype_graph
gc.collect()

# Evolving Convnets

In [ ]:
prototypes = [prototype]

In [ ]:
population,_,_ = convevo.load_population("testing/color_quad_run.xml", False)
prototypes = population[:5]
print(len(prototypes))

In [ ]:
mutate_seed = random.randint(1, 100000)
print("Mutate Seed:", mutate_seed)
mutate_entropy = random.Random(mutate_seed)
eval_seed = random.randint(1, 100000)
print("Eval Seed:", eval_seed)
eval_entropy = random.Random(eval_seed)

population_size = 5
generations = 1
batch_size = 100

breed_options = {
    "input_shape": batch_input_shape(batch_size, data_files["image_size"])
}

evaluator = make_eval(
    batch_size=batch_size, eval_steps=40000, valid_steps=1000, regress_factor=1000.0,
    reuse_cross=True, metric=train_accuracy_metric, entropy=eval_entropy
)
charles = darwin.Darwin(convevo.serialize, evaluator, convevo.breed)
charles.init_population(prototypes, population_size, True, breed_options, mutate_entropy)

for g in range(generations):
    print("Generation", g)
    results = charles.evaluate(eval_entropy)
    convevo.output_results(results, "temp")
    charles.repopulate(population_size, 0.7, 3, results, breed_options, mutate_entropy)

In [ ]:
history,_,_ = convevo.load_population("testing/depth_restart4.xml", True)
print(len(history))
results,_,_ = convevo.load_population("temp/2016-05-21~12_09_54_779.xml", True)
print(len(results))

In [ ]:
mutate_seed = random.randint(1, 100000)
print("Mutate Seed:", mutate_seed)
mutate_entropy = random.Random(mutate_seed)
eval_seed = random.randint(1, 100000)
print("Eval Seed:", eval_seed)
eval_entropy = random.Random(eval_seed)

population_size = 20
generations = 10
batch_size = 100

breed_options = {
    "input_shape": batch_input_shape(batch_size, data_files["image_size"])
}

evaluator = make_eval(
    batch_size=batch_size, eval_steps=2000000, valid_steps=5000,
    reuse_cross=True, metric=train_accuracy_metric, entropy=eval_entropy
)
charles = darwin.Darwin(convevo.serialize, evaluator, convevo.breed)
charles.load_history(history)

for g in range(generations):
    print("Generation", g)
    restart_darwin.repopulate(population_size, 0.25, 4, results, breed_options, mutate_entropy)
    results = restart_darwin.evaluate(eval_entropy)
    convevo.output_results(results, "temp")

In [ ]:
results = darwin.descending_score(charles.history.values())
convevo.output_results(results, "testing", "linear_buckets_run.xml", mutate_seed, eval_seed)
len(results)